In [ ]:
import os

In [ ]:
### replace with necessary path to val dir
SOURCE_DIR = "../../runs/val/nearTarget/nearTarget_38MSteps_8G14P/"
assert os.path.isdir(SOURCE_DIR)

CKPT_DIR = os.path.join(SOURCE_DIR, "data")
assert os.path.isdir(CKPT_DIR)

TRAIN_LOG_PATH = os.path.join(SOURCE_DIR, "train.log")
assert os.path.exists(TRAIN_LOG_PATH)

In [ ]:
"""mono and monoFromMem losses"""
ckpt2mono_lastStep_dct = {}
ckpt2monoFromMem_lastStep_dct = {}
ckpt_number = -1
look_for_newCkpt = True
with open(TRAIN_LOG_PATH, "r") as fo:
    for line in fo:
        if look_for_newCkpt and (len(line.split(" ")) >= 3) and (line.split(" ")[2] == "Mono") and (line.split(" ")[7] == "last") and (line.split(" ")[9] == "---"):
            ckpt2mono_lastStep_dct[ckpt_number] = [float(line.split(" ")[11][:-1]), float(line.split(" ")[13][:-1])]
    
        if look_for_newCkpt and (len(line.split(" ")) >= 3) and (line.split(" ")[2] == "MonoFromMem") and (line.split(" ")[7] == "last") and (line.split(" ")[9] == "---"):
            look_for_newCkpt = False
            ckpt2monoFromMem_lastStep_dct[ckpt_number] = [float(line.split(" ")[11][:-1]), float(line.split(" ")[13][:-1])]
                
        if (not look_for_newCkpt) and (len(line.split(" ")) >= 4) and (line.split(" ")[2] == "=======current_ckpt:"):
            look_for_newCkpt = True
            ckpt_number = int(line.split(" ")[-1].split("=======")[0].split("/")[-1].split(".")[1])            

In [ ]:
ckpt_numbers = []
missing_key = -1
for (dirpath, dirnames, filenames) in os.walk(CKPT_DIR):
    ckpt_numbers.extend(filenames)
    break
    
for i in range(len(ckpt_numbers)):
    ckpt_numbers[i] = int(ckpt_numbers[i].split(".")[1])
    
for key in ckpt_numbers:
    if key not in list(ckpt2mono_lastStep_dct.keys()):
        missing_key = key
        
ckpt2mono_lastStep_dct_final = {}
ckpt2monoFromMem_lastStep_dct_final = {}
for key in ckpt2mono_lastStep_dct:
    if key == -1:
        ckpt2mono_lastStep_dct_final[missing_key] = ckpt2mono_lastStep_dct[key]
        ckpt2monoFromMem_lastStep_dct_final[missing_key] = ckpt2monoFromMem_lastStep_dct[key]
    else:
        ckpt2mono_lastStep_dct_final[key] = ckpt2mono_lastStep_dct[key]
        ckpt2monoFromMem_lastStep_dct_final[key] = ckpt2monoFromMem_lastStep_dct[key]

In [ ]:
min_mono_lastStep = [float("inf"), 0]
min_monoFromMem_lastStep = [float("inf"), 0]

for key, value in ckpt2mono_lastStep_dct_final.items():
    if value[0] < min_mono_lastStep[0]:
        min_mono_lastStep = value
        bestCkpt_mono_lastStep = "ckpt." + str(key) + ".pth"
        
    if ckpt2monoFromMem_lastStep_dct_final[key][0] < min_monoFromMem_lastStep[0]:
        min_monoFromMem_lastStep = ckpt2monoFromMem_lastStep_dct_final[key]
        bestCkpt_monoFromMem_lastStep = "ckpt." + str(key) + ".pth"  
        
print("best validation checkpoint: ", bestCkpt_mono_lastStep,
      ", mono_lastStep: mean -- {}, std -- {}".format(min_mono_lastStep[0],
                                                      min_mono_lastStep[1]))
print("best validation checkpoint: ", bestCkpt_monoFromMem_lastStep,
      ", monoFromMem_lastStep: mean -- {}, std -- {}".format(min_monoFromMem_lastStep[0],
                                                             min_monoFromMem_lastStep[1]))